In [1]:
import requests
import json
import pandas as pd
from pathlib import Path
import random
import time
from datetime import datetime
import pytz
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type
import logging

# --- Cấu hình cơ bản ---
logging.basicConfig(
    filename="crawl_reviews.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# --- Các hàm tiện ích (Request, Checkpoint) ---

@retry(
    stop=stop_after_attempt(3),
    wait=wait_fixed(2),
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    after=lambda retry_state: logging.warning(f"Thử lại lần {retry_state.attempt_number} cho URL: {retry_state.args[0]}")
)
def safe_request(url, headers):
    """Thực hiện request HTTP an toàn với cơ chế retry."""
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response

def save_checkpoint(crawled_ids, checkpoint_file):
    """Lưu danh sách các ID đã crawl vào file checkpoint."""
    with open(checkpoint_file, "w") as f:
        json.dump(list(crawled_ids), f)
    logging.info(f"Đã lưu checkpoint vào {checkpoint_file}")


def load_checkpoint(checkpoint_file):
    """Tải danh sách các ID đã crawl từ file checkpoint."""
    try:
        with open(checkpoint_file, "r") as f:
            return set(json.load(f))
    except FileNotFoundError:
        return set()

# --- Các hàm xử lý và chuyển đổi dữ liệu ---

def map_json_to_reviews(json_data):
    """Chuyển đổi dữ liệu JSON review thành danh sách các dictionary."""
    reviews = []
    for review in json_data.get("data", []):
        created_by = review.get("created_by", {})
        review_data = {
            "id": review.get("id"),
            "customer_id": created_by.get("id"),
            "product_id": review.get("product_id"),
            "rating": review.get("rating", 0),
            "title": review.get("title", ""),
            "content": review.get("content", ""),
            "thank_count": review.get("thank_count", 0),
            "comment_count": review.get("comment_count", 0),
        }
        reviews.append(review_data)
    return reviews

def map_json_to_customers(json_data):
    """Chuyển đổi dữ liệu JSON review để lấy thông tin khách hàng."""
    customers = {}
    for review in json_data.get("data", []):
        created_by = review.get("created_by", {})
        customer_id = created_by.get("id")
        if customer_id and customer_id not in customers:
            customers[customer_id] = {
                "id": customer_id,
                "full_name": created_by.get("full_name", ""),
                "avatar_url": created_by.get("avatar_url", ""),
                "purchased": created_by.get("purchased", False),
            }
    return list(customers.values())

def save_data_to_csv(data, file_path):
    """Lưu danh sách dữ liệu vào file CSV."""
    if not data:
        return
    df = pd.DataFrame(data)
    # Ghi đè nếu file không tồn tại hoặc ghi nối tiếp nếu file đã có
    if not Path(file_path).exists():
        df.to_csv(file_path, index=False, encoding="utf-8-sig")
    else:
        df.to_csv(file_path, mode="a", header=False, index=False, encoding="utf-8-sig")
    logging.info(f"Đã lưu {len(data)} dòng vào file {file_path}")

# --- Hàm chính để thu thập dữ liệu ---

def fetch_reviews_for_products(product_ids, output_folder):
    """
    Thu thập tất cả review và thông tin khách hàng cho một danh sách ID sản phẩm.
    """
    api_url = "https://tiki.vn/api/v2/reviews?product_id={}&page={}&limit=20"
    reviews_file = Path(output_folder) / "reviews.csv"
    customers_file = Path(output_folder) / "customers.csv"
    checkpoint_file = Path(output_folder) / "reviews_checkpoint.json"

    crawled_product_ids = load_checkpoint(checkpoint_file)

    for product_id in product_ids:
        if product_id in crawled_product_ids:
            logging.info(f"Bỏ qua sản phẩm đã crawl review: {product_id}")
            continue

        page = 1
        total_reviews_fetched = 0
        while True:
            logging.info(f"Đang crawl review sản phẩm {product_id}, trang {page}...")
            try:
                response = safe_request(api_url.format(product_id, page), headers=headers)
                data = response.json()
                
                # Trích xuất và lưu dữ liệu
                reviews = map_json_to_reviews(data)
                customers = map_json_to_customers(data)

                if not reviews:
                    logging.info(f"Hoàn thành crawl sản phẩm {product_id}. Tổng số review: {total_reviews_fetched}")
                    break

                save_data_to_csv(reviews, reviews_file)
                save_data_to_csv(customers, customers_file)

                total_reviews_fetched += len(reviews)
                page += 1
                time.sleep(random.uniform(0.5, 1.5))

            except Exception as e:
                logging.error(f"Lỗi khi crawl review sản phẩm {product_id}, trang {page}: {e}")
                break # Dừng crawl sản phẩm này nếu có lỗi

        # Đánh dấu sản phẩm này đã được crawl và lưu checkpoint
        crawled_product_ids.add(product_id)
        save_checkpoint(crawled_product_ids, checkpoint_file)

def crawl_category_recursively(category_id, output_folder):
    """
    Đệ quy để crawl các danh mục, lấy ID sản phẩm và sau đó lấy review.
    """
    logging.info(f"Bắt đầu crawl danh mục ID: {category_id}")
    
    # 1. Lấy ID sản phẩm trong danh mục hiện tại
    product_ids = []
    page = 1
    while True:
        try:
            api_url = f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&page={page}&category={category_id}"
            response = safe_request(api_url, headers=headers)
            products_data = response.json().get("data", [])
            if not products_data:
                logging.info(f"Không còn sản phẩm nào trong danh mục {category_id} ở trang {page}.")
                break
            
            for product in products_data:
                product_ids.append(str(product["id"]))
            
            page += 1
            time.sleep(random.uniform(0.5, 1))

        except Exception as e:
            logging.error(f"Lỗi khi lấy ID sản phẩm từ danh mục {category_id}, trang {page}: {e}")
            break
    
    # 2. Lấy reviews cho các sản phẩm vừa tìm được
    if product_ids:
        logging.info(f"Tìm thấy {len(product_ids)} sản phẩm trong danh mục {category_id}. Bắt đầu lấy reviews.")
        fetch_reviews_for_products(product_ids, output_folder)

    # 3. Tìm và đệ quy vào các danh mục con
    try:
        children_api = f"https://tiki.vn/api/v2/categories?parent_id={category_id}"
        response = safe_request(children_api, headers=headers)
        children_categories = response.json().get("data", [])
        for child_cat in children_categories:
            crawl_category_recursively(child_cat['id'], output_folder)
    except Exception as e:
        logging.error(f"Không thể lấy danh mục con của {category_id}: {e}")

# --- Điểm bắt đầu của chương trình ---
if __name__ == "__main__":
    # Thay đổi CATEGORY_NAME và CATEGORY_ID tùy theo nhu cầu
    CATEGORY_NAME = "nha-sach-tiki"
    CATEGORY_ID = "8322"

    # Tạo thư mục đầu ra
    output_folder = Path(f"./data/{CATEGORY_NAME}")
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Bắt đầu quá trình crawl
    crawl_category_recursively(CATEGORY_ID, output_folder)
    
    logging.info("Hoàn tất quá trình crawl.")


KeyboardInterrupt: 